## Prerequisites for unix

In [1]:
!apt-get -y install openjdk-8-jdk-headless
!wget http://apache.osuosl.org/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install findspark kaggle

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk-headless is already the newest version (8u191-b12-2ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
--2019-04-10 16:20:57--  http://apache.osuosl.org/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 64.50.236.52, 64.50.233.100, 140.211.166.134, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|64.50.236.52|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230778742 (220M) [application/x-gzip]
Saving to: ‘spark-2.4.1-bin-hadoop2.7.tgz’

spark-2.4.1-bin-had 100%[===================>] 220.09M  50.2MB/s    in 5.0s    

2019-04-10 16:21:07 (44.1 MB/s) - ‘spark-2.4.1-bin-hadoop2.7.tgz’ saved [230778742/230778742]



In [2]:
import os

# setup environment variables
os.environ["JAVA_HOME"] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ["SPARK_HOME"] = '/content/spark-2.4.1-bin-hadoop2.7'
os.environ['KAGGLE_USERNAME'] = 'ronaldsumbayak'
os.environ['KAGGLE_KEY'] = '771009dffffa0cd0883d5fb98594d756'

# verify
print(os.environ['JAVA_HOME'])
print(os.environ['SPARK_HOME'])
print(os.environ['KAGGLE_USERNAME'])
print(os.environ['KAGGLE_KEY'])

/usr/lib/jvm/java-8-openjdk-amd64
/content/spark-2.4.1-bin-hadoop2.7
ronaldsumbayak
771009dffffa0cd0883d5fb98594d756


## Data Preparation

In [3]:
!kaggle datasets download -d olistbr/brazilian-ecommerce
!unzip brazilian-ecommerce.zip -d data

 64% 27.0M/42.4M [00:00<00:00, 143MB/s]
100% 42.4M/42.4M [00:00<00:00, 142MB/s]
Archive:  brazilian-ecommerce.zip
  inflating: data/olist_customers_dataset.csv  
  inflating: data/olist_sellers_dataset.csv  
  inflating: data/olist_order_reviews_dataset.csv  
  inflating: data/olist_order_payments_dataset.csv  
  inflating: data/product_category_name_translation.csv  
  inflating: data/olist_products_dataset.csv  
  inflating: data/olist_order_items_dataset.csv  
  inflating: data/olist_orders_dataset.csv  
  inflating: data/olist_geolocation_dataset.csv  


## Spark Setup

In [0]:
import findspark
findspark.init()

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

## Loading Data

In [0]:
orders = spark.read.csv('data/olist_orders_dataset.csv', header=True)
order_items = spark.read.csv('data/olist_order_items_dataset.csv', header=True)
products = spark.read.csv('data/olist_products_dataset.csv', header=True)

In [7]:
print('Orders      :  {} rows'.format(orders.count()))
print('Order Items : {} rows'.format(order_items.count()))
print('Products    :  {} rows'.format(products.count()))

Orders      :  99441 rows
Order Items : 112650 rows
Products    :  32951 rows


In [8]:
orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_carrier_date: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)
 |-- order_estimated_delivery_date: string (nullable = true)



In [9]:
order_items.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- freight_value: string (nullable = true)



In [10]:
products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: string (nullable = true)
 |-- product_description_lenght: string (nullable = true)
 |-- product_photos_qty: string (nullable = true)
 |-- product_weight_g: string (nullable = true)
 |-- product_length_cm: string (nullable = true)
 |-- product_height_cm: string (nullable = true)
 |-- product_width_cm: string (nullable = true)



In [11]:
orders.show(10, truncate=False)

+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297304e76186b10a928d|delivered   |2017-10-02 10:56:33     |2017-10-02 11:07:15|2017-10-04 19:55:00         |2017-10-10 21:25:13          |2017-10-18 00:00:00          |
|53cdb2fc8bc7dce0b6741e2150273451|b0830fb4747a6c6d20dea0b8c802d7ef|delivered   |2018-07-24 20:41:37     |2018-07-26 03:24:27|2018-07-26 14:3

In [12]:
order_items.show(10, truncate=False)

+--------------------------------+-------------+--------------------------------+--------------------------------+-------------------+------+-------------+
|order_id                        |order_item_id|product_id                      |seller_id                       |shipping_limit_date|price |freight_value|
+--------------------------------+-------------+--------------------------------+--------------------------------+-------------------+------+-------------+
|00010242fe8c5a6d1ba2dd792cb16214|1            |4244733e06e7ecb4970a6e2683c13e61|48436dade18ac8b2bce089ec2a041202|2017-09-19 09:45:35|58.90 |13.29        |
|00018f77f2f0320c557190d7a144bdd3|1            |e5f2d52b802189ee658865ca93d83a8f|dd7ddc04e1b6c2c614352b383efe2d36|2017-05-03 11:05:13|239.90|19.93        |
|000229ec398224ef6ca0657da4fc703e|1            |c777355d18b72b67abbeef9df44fd0fd|5b51032eddd242adc84c38acab88f23d|2018-01-18 14:48:30|199.00|17.87        |
|00024acbcdf0a6daa1e931b038114c75|1            |7634da152a4610f1

In [13]:
products.show(10, truncate=False)

+--------------------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|product_id                      |product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff4541ed26657ea517e5|perfumaria           |40                 |287                       |1                 |225             |16               |10               |14              |
|3aa071139cb16b67ca9e5dea641aaa2f|artes                |44                 |276                       |1                 |1000            |30               |18               |20              |
|96bd76ec8810374ed1b65e291975717f|e

## Preprocessing

In [0]:
# Remove duplicate pair of (order_id, product_id) to avoid
# error FPGrowth algorithm can't take duplicate
# item in the same row
distinct_order_items = order_items.select('order_id', 'product_id').distinct()

In [15]:
distinct_order_items.count()

102425

In [0]:
import pyspark.sql.functions as F

In [0]:
# Group items of same order together
data = (distinct_order_items.groupby('order_id')
                            .agg(F.collect_list('product_id').alias('items')))

In [18]:
data.count()

98666

In [19]:
data.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [20]:
data.show(truncate=False)

+--------------------------------+--------------------------------------------------------------------+
|order_id                        |items                                                               |
+--------------------------------+--------------------------------------------------------------------+
|014405982914c2cde2796ddcf0b8703d|[e95ee6822b66ac6058e2e4aff656071a, 6782d593f63105318f46bbf7633279bf]|
|019886de8f385a39b75bedbb726fd4ef|[e9a69340883a438c3f91739d14d3a56d]                                  |
|01a6ad782455876aa89081449d49c452|[036734b5a58d5d4f46b0616ddc047ced]                                  |
|01d907b3e209269e120a365fc2b97524|[b1434a8f79cb3528540d9b21e686e823]                                  |
|028dc52e12ddda803ec1e35eb0b7b0d9|[d86a6c48f83b045cbba6df84926a1f25]                                  |
|036dd381dfb3ec75e0a63e14828cc871|[aa8d88eb4b9cb38894e33fa624c4287f]                                  |
|03ebfa9712b7dbc7031291856263b314|[aa6746e94490239d3d9ee6ab89779

## FPGrowth

In [0]:
from pyspark.ml.fpm import FPGrowth

### Utility functions

In [0]:
import random


def get_random_items(model, n):
    """
    Get random itemsets from FPGrowth model's
    association rules to make sure we
    get items that has prediction.
    """
    items = model.associationRules \
                 .select('antecedent') \
                 .orderBy(F.rand()) \
                 .limit(n) \
                 .collect()
    return list(map(lambda x: x[0], items))


def get_random_products(n, max_items):
    '''
    Get random itemsets from products table
    '''
    itemsets = list()
    for i in range(n):
        items = products.select('product_id') \
                        .orderBy(F.rand()) \
                        .limit(random.randint(1, max_items)) \
                        .collect()
        itemsets.append(list(map(lambda x: x[0], items)))
    return itemsets

### Model 1 (minSupport: 0.3, minConfidence: 0.8) (default values)

In [0]:
fp = FPGrowth()

In [0]:
fpm = fp.fit(data)

In [25]:
fpm.freqItemsets.show(truncate=False)

+-----+----+
|items|freq|
+-----+----+
+-----+----+



In [26]:
fpm.associationRules.show(truncate=False)

+----------+----------+----------+----+
|antecedent|consequent|confidence|lift|
+----------+----------+----------+----+
+----------+----------+----------+----+



In [0]:
# Crate fake orders for testing
# Association rules is empty. So we use #get_random_products
fake_orders_n = 20
fake_orders_id = range(1, fake_orders_n+1)
fake_orders_items = get_random_products(fake_orders_n, 3)
fake_orders = list(zip(fake_orders_id, fake_orders_items))

In [0]:
# Convert to spark DataFrame
test_data = spark.createDataFrame(fake_orders, ['order_id', 'items'])

In [29]:
test_data.printSchema()

root
 |-- order_id: long (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [30]:
# final test DataFrame
test_data.show(truncate=False)

+--------+------------------------------------------------------------------------------------------------------+
|order_id|items                                                                                                 |
+--------+------------------------------------------------------------------------------------------------------+
|1       |[89321f94e35fc6d7903d36f74e351d40, 1a0bba83632b7dc923d939a7fa2c4530]                                  |
|2       |[853ee0759242b6c1e8e1de5052df3fc4, 96627b65a0070635b620d07f97e63347, ef5ad80d84199def6971cac1eec1bd0e]|
|3       |[c7963f30536d39f982fe6af66ab3e89b]                                                                    |
|4       |[44a064a339782c3cdd7e457b169ef0b0]                                                                    |
|5       |[0e35a413d832e63c5d46101b7c88de0f, 6c1a3375a7b0cb7b5a454e123c61311b]                                  |
|6       |[57f202d5e0ed7a3a2d75e479e1579cfe]                                            

In [0]:
predictions = fpm.transform(test_data)

In [32]:
# Schema of products data does not contains
# product name. So we only show it by
# product's id
predictions.show(truncate=False)

+--------+------------------------------------------------------------------------------------------------------+----------+
|order_id|items                                                                                                 |prediction|
+--------+------------------------------------------------------------------------------------------------------+----------+
|1       |[89321f94e35fc6d7903d36f74e351d40, 1a0bba83632b7dc923d939a7fa2c4530]                                  |[]        |
|2       |[853ee0759242b6c1e8e1de5052df3fc4, 96627b65a0070635b620d07f97e63347, ef5ad80d84199def6971cac1eec1bd0e]|[]        |
|3       |[c7963f30536d39f982fe6af66ab3e89b]                                                                    |[]        |
|4       |[44a064a339782c3cdd7e457b169ef0b0]                                                                    |[]        |
|5       |[0e35a413d832e63c5d46101b7c88de0f, 6c1a3375a7b0cb7b5a454e123c61311b]                                  |[]        |


This model does not produce any predictions as expected. We will try lowering `minConfidence` and `minSupport`

### Model 2 (minConfidence: 0.03, minSupport: 0.08)

In [0]:
fp = FPGrowth(minConfidence=0.03, minSupport=0.08)

In [0]:
fpm = fp.fit(data)

In [35]:
fpm.freqItemsets.show(truncate=False)

+-----+----+
|items|freq|
+-----+----+
+-----+----+



In [36]:
fpm.associationRules.show(truncate=False)

+----------+----------+----------+----+
|antecedent|consequent|confidence|lift|
+----------+----------+----------+----+
+----------+----------+----------+----+



No association rules yet. Not worth a test. Try lower values!

### Model 3 (minConfidence: 0.003, minSupport: 0.008)

In [0]:
fp = FPGrowth(minConfidence=0.003, minSupport=0.008)

In [0]:
fpm = fp.fit(data)

In [39]:
fpm.freqItemsets.show(truncate=False)

+-----+----+
|items|freq|
+-----+----+
+-----+----+



In [40]:
fpm.associationRules.show(truncate=False)

+----------+----------+----------+----+
|antecedent|consequent|confidence|lift|
+----------+----------+----------+----+
+----------+----------+----------+----+



Still no association rules. Keep lowering the values!

### Model 4 (minConfidence: 0.00003, minSupport: 0.00008)

In [0]:
fp = FPGrowth(minConfidence=0.00003, minSupport=0.00008)

In [0]:
fpm = fp.fit(data)

In [43]:
fpm.freqItemsets.show(truncate=False)

+----------------------------------+----+
|items                             |freq|
+----------------------------------+----+
|[a4aa7c1427c31344e5f7cc3d839fe562]|42  |
|[b60856ce32d90658dbf99b9485327c25]|26  |
|[33fac76e7f17367dedaef7954bbb33b4]|20  |
|[03bb06cda40712fb8473f7962fb7d198]|8   |
|[20a64c9ab614812a2634ebbd69bb4a02]|8   |
|[5e53af05c040b01584b81b3549945faf]|16  |
|[be1c4cf8a2d2aade623eb7a646d2a0bc]|14  |
|[f89cd865cac300a9bf1320dd8f0fa223]|12  |
|[00878d953636afec00d3e85d55a12e7f]|11  |
|[2ff17002562478fb03cd44f09e7ca51a]|10  |
|[11fb16e86df4d50cd2c289ede0b70f42]|9   |
|[99a4788cb24856965c36a24e339b6058]|467 |
|[b0c89945c034268074f5f80b362bda34]|16  |
|[9af7b4a3262141c12ba9bd489a8573c7]|14  |
|[6fd08d44046ab994b96ff38ad6fcfba1]|12  |
|[2827ebfb043a328a13a06ca9165cad0a]|11  |
|[293ee8ac0a45f559cdeba37b938fdb66]|10  |
|[aca2eb7d00ea1a7b8ebd4e68314663af]|431 |
|[31a2f42a87890f87d77daebdfabc182e]|42  |
|[e85d91619b497e98754390b6198a9818]|26  |
+---------------------------------

In [44]:
fpm.associationRules.show(truncate=False)

+----------------------------------+----------------------------------+--------------------+------------------+
|antecedent                        |consequent                        |confidence          |lift              |
+----------------------------------+----------------------------------+--------------------+------------------+
|[3f14d740544f37ece8a9e7bc8349797e]|[36f60d45225e60c7da4558b070ce4b60]|0.14814814814814814 |131.68635301968632|
|[35afc973633aaeb6b877ff57b2793310]|[99a4788cb24856965c36a24e339b6058]|0.1858974358974359  |39.27570965793664 |
|[99a4788cb24856965c36a24e339b6058]|[35afc973633aaeb6b877ff57b2793310]|0.06209850107066381 |39.27570965793664 |
|[e53e557d5a159f5aa2c5e995dfdf244b]|[36f60d45225e60c7da4558b070ce4b60]|0.21794871794871795 |193.73088473088472|
|[53759a2ecddad2bb87a079a1f1519f73]|[389d119b48cf3043d311335e499d9c6b]|0.0313588850174216  |9.94873231231164  |
|[53759a2ecddad2bb87a079a1f1519f73]|[368c6c730842d78016ad823897a372db]|0.027874564459930314|9.4511057628

In [45]:
fpm.associationRules.count()

14

We starting to get association rules. We will test the model against itemsets from its own association rules.

In [0]:
# Crate fake orders for testing
# We can now get fake orders from
# model's association rules
fake_orders_n = 20
fake_orders_id = range(1, fake_orders_n+1)
fake_orders_items = get_random_items(fpm, fake_orders_n)
fake_orders = list(zip(fake_orders_id, fake_orders_items))

In [0]:
test_data = spark.createDataFrame(fake_orders, ['order_id', 'items'])

In [48]:
test_data.show(truncate=False)

+--------+----------------------------------+
|order_id|items                             |
+--------+----------------------------------+
|1       |[389d119b48cf3043d311335e499d9c6b]|
|2       |[99a4788cb24856965c36a24e339b6058]|
|3       |[422879e10f46682990de24d770e7f83d]|
|4       |[36f60d45225e60c7da4558b070ce4b60]|
|5       |[e53e557d5a159f5aa2c5e995dfdf244b]|
|6       |[389d119b48cf3043d311335e499d9c6b]|
|7       |[36f60d45225e60c7da4558b070ce4b60]|
|8       |[4fcb3d9a5f4871e8362dfedbdb02b064]|
|9       |[53759a2ecddad2bb87a079a1f1519f73]|
|10      |[368c6c730842d78016ad823897a372db]|
|11      |[35afc973633aaeb6b877ff57b2793310]|
|12      |[3f14d740544f37ece8a9e7bc8349797e]|
|13      |[f4f67ccaece962d013a4e1d7dc3a61f7]|
|14      |[53759a2ecddad2bb87a079a1f1519f73]|
+--------+----------------------------------+



In [0]:
predictions = fpm.transform(test_data)

In [50]:
predictions.show(truncate=False)

+--------+----------------------------------+--------------------------------------------------------------------+
|order_id|items                             |prediction                                                          |
+--------+----------------------------------+--------------------------------------------------------------------+
|1       |[389d119b48cf3043d311335e499d9c6b]|[422879e10f46682990de24d770e7f83d, 53759a2ecddad2bb87a079a1f1519f73]|
|2       |[99a4788cb24856965c36a24e339b6058]|[35afc973633aaeb6b877ff57b2793310]                                  |
|3       |[422879e10f46682990de24d770e7f83d]|[389d119b48cf3043d311335e499d9c6b]                                  |
|4       |[36f60d45225e60c7da4558b070ce4b60]|[e53e557d5a159f5aa2c5e995dfdf244b, 3f14d740544f37ece8a9e7bc8349797e]|
|5       |[e53e557d5a159f5aa2c5e995dfdf244b]|[36f60d45225e60c7da4558b070ce4b60]                                  |
|6       |[389d119b48cf3043d311335e499d9c6b]|[422879e10f46682990de24d770e7f83d, 

We starting to get a proper predictions. Let's try lowering `minConfidence` and `minSupport` more.

### Model 5 (minConfidence: 0.000003, minSupport: 0.000008)

In [0]:
fp = FPGrowth(minConfidence=0.000003, minSupport=0.000008)

In [0]:
fpm = fp.fit(data)

In [53]:
fpm.freqItemsets.show(truncate=False)

+--------------------------------------------------------------------+----+
|items                                                               |freq|
+--------------------------------------------------------------------+----+
|[c647d965c3bd45fa151a61dc4233b7cc]                                  |1   |
|[a4aa7c1427c31344e5f7cc3d839fe562]                                  |42  |
|[00989337a1916a0055eedd1fdb35eb53]                                  |2   |
|[aa59c9e7a89f70e05e190130174cd4aa]                                  |1   |
|[d1a7aa885550f1d6803bd8edff173851]                                  |1   |
|[d1a7aa885550f1d6803bd8edff173851, bb32b03270ad3f5a8aa9474d354119ab]|1   |
|[304fad8dc4d2012dc4062839972f2d96]                                  |2   |
|[49bfa76ac7062b8b5a0bf9601623d612]                                  |2   |
|[7d02633fd7494877cf1dde4210bdab8f]                                  |1   |
|[632c37951cab36a4e928428cbc4fa4f4]                                  |5   |
|[b42e79652d

In [54]:
fpm.associationRules.show(truncate=False)

+--------------------------------------------------------------------+----------------------------------+--------------------+------------------+
|antecedent                                                          |consequent                        |confidence          |lift              |
+--------------------------------------------------------------------+----------------------------------+--------------------+------------------+
|[1de1332e38dc2f575edfeae165edea4e]                                  |[ba131cee7989ef04b93a79030969c92f]|0.14285714285714285 |3523.7857142857138|
|[43a888fc6acdffc23011578f63d5f045]                                  |[8e87f0e6eb7e5a27568309035f4fc854]|1.0                 |24666.5           |
|[0b3f6a2323c814e1b19fd44838c160ef]                                  |[6df3e292ba834323d4d8164d4c88e698]|0.25                |4933.3            |
|[c4baedd846ed09b85f78a781b522f126]                                  |[d1c427060a0f73f6b889a5c7c61f2ac4]|0.02097902097902098

In [55]:
fpm.associationRules.count()

14254

This model seems to have more association rules than the previous one

In [0]:
# Crate fake orders for testing
fake_orders_n = 20
fake_orders_id = range(1, fake_orders_n+1)
fake_orders_items = get_random_items(fpm, fake_orders_n)
fake_orders = list(zip(fake_orders_id, fake_orders_items))

In [0]:
test_data = spark.createDataFrame(fake_orders, ['order_id', 'items'])

In [58]:
test_data.show(truncate=False)

+--------+----------------------------------------------------------------------------------------------------------------------------------------+
|order_id|items                                                                                                                                   |
+--------+----------------------------------------------------------------------------------------------------------------------------------------+
|1       |[dde35467201d3ff9e3f8e4297aa2768e]                                                                                                      |
|2       |[168b1b45c68037b0490157ee10540349]                                                                                                      |
|3       |[8c591ab0ca519558779df02023177f44]                                                                                                      |
|4       |[4b96786612ebe7463132fce2c4dca136]                                                                    

In [0]:
predictions = fpm.transform(test_data)

In [60]:
predictions.show(truncate=False)

+--------+----------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|order_id|items                                                                                                                                   |prediction                                                                                                                                                                                        

## Conclusion

Lower `minConfidence` and `minSupport` values can discover more association rules, but may get predictions that have lower confidence value.

Easier to read version: [big-data-frequent-itemsets](https://colab.research.google.com/drive/1tmGcdJ9Yza_zA-hETMrpT-lJymWXKvZm)